In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
# plt.style.use('fivethirtyeight')
# 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.style.use('ggplot')
sns.set(style='whitegrid',color_codes=True)
%matplotlib inline  


# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
plt.rcParams['axes.unicode_minus'] = False

# matplotlib 한글 폰트 출력코드
from matplotlib import font_manager, rc
import platform
try : 
    if platform.system() == 'Windows':
    # 윈도우인 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    else:    
    # Mac 인 경우
        rc('font', family='AppleGothic')
except : 
    pass


import warnings
warnings.filterwarnings('ignore')

import time, datetime


In [2]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
train = pd.read_csv('중급자_train.csv')
test = pd.read_csv('중급자_test.csv')


In [4]:
# searchfor = ['공항', '병원','공단','아파트','시청','터미널','시장']

# a['s_name'][a['s_name'].str.contains('|'.join(searchfor))]

# a= pd.DataFrame(train['station_name'].value_counts())
# a=a.reset_index()
# a.columns=['s_name','geton_off']
# a

In [5]:
# b=a[a['geton_off']>=500]['station_name']

# a= pd.DataFrame(train['station_name'].value_counts())
# a=a.reset_index()
# a.columns=['s_name','geton_off']
# a
# all = ['공항', '병원','공단','아파트','시청','터미널','시장','오거리','동']

# train[train['station_name'].str.contains('|'.join(all))]

# hospital = b[b.str.contains('병원')]

# school = b[b.str.contains('학교')]



### Start

### 1. 합계와 평균 변수 생성 


In [6]:
train['sum']=train['6_8_ride']+train['8_10_ride']+train['10_12_ride']+train['6_8_off']+train['8_10_off']+train['10_12_off']
train['avg']=train['sum']/12

In [7]:
station_sum_top=train.groupby(['station_code','station_name'])['sum','avg'].sum().sort_values(['sum'],ascending=False)

In [126]:
#상위 100개 
pd.set_option('display.max_rows', 100)
station_sum_top.head(100)

,station_code,station_name,sum,avg
0,3270,제주시청(아라방면),34695.0,2891.250000
1,3271,제주시청(광양방면),31482.0,2623.500000
2,357,한라병원,30438.0,2536.500000
3,2972,제주국제공항(종점),26776.0,2231.333333
4,358,한라병원,26076.0,2173.000000
5,1564,제주시외버스터미널,24234.0,2019.500000
6,149,제주버스터미널,21855.0,1821.250000
7,1957,중앙로터리,19569.0,1630.750000
8,638,제주국제공항(구제주방면),18994.0,1582.833333
9,150,제주버스터미널,18746.0,1562.166667


In [9]:
#인덱스를 칼럼으로 되돌리기
station_sum_top=station_sum_top.reset_index(['station_code','station_name'])

In [10]:
# 저장 및 불러오기
station_sum_top.to_csv('station_sum_top.csv',index=False)
station_sum_top=pd.read_csv('station_sum_top.csv')

### 2. 리스트에 있는 단어들로 데이터 프레임 만들기 
### Train 데이터프레임으로 돌려야한다!!!!!!!!

In [11]:
#['학교', '공항', '터미널', '오거리', '병원', '아파트', '환승', '로터리','중앙','마트','제주','시장','사거리','약국','공단']
name_list=station_sum_top['station_name'].tolist()
# name_list

In [200]:
name_keyword=[ '공항','터미널','오거리','병원','중앙','제주','시장','광양','버스']
# name_keyword

In [201]:
#리스트에 있는 단어 1 없으면 0
train['have_keyword'] = np.where(train['station_name'].str.contains('|'.join(name_keyword)),1,0)

In [202]:
# keyword 보유로 df 분리함
have_keyword_df = train[train['have_keyword']== 1]
not_keyword_df = train[train['have_keyword']== 0]

In [203]:
#shape 확인
print(have_keyword_df.shape)
print(not_keyword_df.shape)


(85700, 52)
(329723, 52)


### 3. 상관계수 및 회귀계수 그려보기

In [204]:
c = train[['have_keyword','18~20_ride']].corr()
c  #

,have_keyword,18~20_ride
have_keyword,1.000000,0.190942
18~20_ride,0.190942,1.000000


In [177]:
b = train[['have_keyword','18~20_ride']].corr()
b #[ '공항','터미널','오거리','병원', '로터리','중앙','제주','시장','광양','버스']

,have_keyword,18~20_ride
have_keyword,1.000000,0.189279
18~20_ride,0.189279,1.000000


In [81]:
a = train[['have_keyword','18~20_ride']].corr()
a  #[ '공항', '터미널', '오거리', '병원', '아파트', '환승', '로터리','중앙','제주','시장','사거리','공단']

,have_keyword,18~20_ride
have_keyword,1.000000,0.155376
18~20_ride,0.155376,1.000000
